In [10]:
!git clone https://github.com/awslabs/rag-qa-arena
!mkdir rag-qa-benchmarking
!mv rag-qa-arena/data/annotations_science_with_citation.jsonl ./rag-qa-benchmarking/
!rm -rf rag-qa-arena

Cloning into 'rag-qa-arena'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 55 (delta 15), reused 41 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 3.59 MiB | 26.24 MiB/s, done.
Resolving deltas: 100% (15/15), done.
mkdir: cannot create directory ‘rag-qa-benchmarking’: File exists


UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

In [7]:
!curl https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz
!tar -xvzf lotte.tar.gz
! cp lotte/science/test/collection.tsv ./rag-qa-benchmarking/science_test_collection.tsv
! rm lotte.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3410M  100 3410M    0     0  5158k      0  0:11:17  0:11:17 --:--:-- 5240k:08  0:00:32  0:09:36 5210k:29 5331k00:23 5549k


In [3]:
import os
import pandas as pd

In [4]:
questions = pd.read_json("rag-qa-benchmarking/annotations_science_with_citation.jsonl", lines=True)

docs = pd.read_csv(
    "rag-qa-benchmarking/science_test_collection.tsv",
    sep="\t",
    header=None,
)
docs.columns = ["doc_id", "doc_text"]

In [5]:
percentage_to_use = 100
proportion_to_use = percentage_to_use/100
papers_directory = f'rag-qa-benchmarking/lfrqa'
amount_of_docs_to_use = int(len(docs)*proportion_to_use)
partial_docs = docs.head(amount_of_docs_to_use)
print(f"Using {amount_of_docs_to_use} out of {len(docs)} documents")

Using 1694041 out of 1694041 documents


In [6]:
! find rag-qa-benchmarking/lfrqa/science_docs_for_paperqa/files -type f -delete
! rm -rf rag-qa-benchmarking/lfrqa/science_docs_for_paperqa_index

In [7]:
for i, row in partial_docs.iterrows():
    doc_id = row["doc_id"]
    doc_text = row["doc_text"]
    
    with open(f"{papers_directory}/science_docs_for_paperqa/files/{doc_id}.txt", "w", encoding="utf-8") as f:
        f.write(doc_text)
        
    if i % int(len(partial_docs)*0.05) == 0:
        progress = (i+1) / len(partial_docs)
        print(f"Progress: {progress:.2%}")


Progress: 0.00%
Progress: 5.00%
Progress: 10.00%
Progress: 15.00%
Progress: 20.00%
Progress: 25.00%
Progress: 30.00%
Progress: 35.00%
Progress: 40.00%
Progress: 45.00%
Progress: 50.00%
Progress: 55.00%
Progress: 60.00%
Progress: 65.00%
Progress: 70.00%
Progress: 75.00%
Progress: 80.00%
Progress: 85.00%
Progress: 90.00%
Progress: 95.00%
Progress: 100.00%


In [8]:
manifest = partial_docs.copy()
manifest['file_location'] = manifest['doc_id'].apply(lambda x: f"files/{x}.txt")
manifest['doi'] = ""
manifest['overwrite_fields_from_metadata'] = False
manifest['title'] = manifest['doc_id'].apply(lambda x: x)
manifest['key'] = manifest['title']
manifest['docname'] = manifest['title']
manifest['citation'] = "_"
manifest.drop(columns=['doc_id', 'doc_text'], inplace=True)
manifest.to_csv(f"{papers_directory}/science_docs_for_paperqa/manifest.csv", index=False)

In [9]:
partial_questions = questions[
    questions.gold_doc_ids.apply(lambda ids: all(id < amount_of_docs_to_use for id in ids))
]
partial_questions.to_csv(f"{papers_directory}/questions.csv", index=False)